# Демонстрация использования классов `Oscillogram` и `DataAnonymizer`

Этот ноутбук показывает, как использовать класс `Oscillogram` для загрузки и базового анализа осциллограмм COMTRADE, а также класс `DataAnonymizer` для анонимизации данных в файлах `.cfg` и `.dat`.

In [ ]:
import os
import shutil

# Создаем временную директорию для демонстрации
sample_data_dir = "temp_sample_data_anonymizer"
if os.path.exists(sample_data_dir):
    shutil.rmtree(sample_data_dir)
os.makedirs(sample_data_dir)

# Создаем пример файла .cfg (простейшая структура)
sample_cfg_content = """PS_Station,REC001,1999
2,1A,1D
1,АналоговыйСигнал1,A,,V,1.0,0,0,0,100,1,1,P
2,ДискретныйСигнал1,A,,1
50.0
1
1000,1000
01/01/2023,00:00:00.000000
01/01/2023,00:00:01.000000
ASCII
1.0
"""
cfg_file_path = os.path.join(sample_data_dir, "sample_osc_1.cfg")
with open(cfg_file_path, "w", encoding="utf-8") as f:
    f.write(sample_cfg_content)

# Создаем парный файл .dat (простейший)
sample_dat_content = """1,0,10,0
2,1,20,1
3,2,30,0
"""
dat_file_path = os.path.join(sample_data_dir, "sample_osc_1.dat")
with open(dat_file_path, "w", encoding="utf-8") as f:
    f.write(sample_dat_content)

print(f"Созданы демонстрационные файлы в директории: {sample_data_dir}")
print("Содержимое директории:")
for item in os.listdir(sample_data_dir):
    print(f"- {item}")

## 1. Использование `Oscillogram`

Класс `Oscillogram` из `core.oscillogram` является оберткой для данных COMTRADE. Он загружает файлы `.cfg` и `.dat` (используя `comtrade_APS.py` внутри) и предоставляет удобный доступ к метаданным и самим данным осциллограммы.

In [ ]:
import sys
import os
# Добавляем корневую директорию проекта в PYTHONPATH, чтобы работали импорты модулей проекта
# Это может потребовать корректировки в зависимости от того, откуда запускается ноутбук
# Если ноутбук в notebooks/, то ../ добавляет корень проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram
import pandas as pd # для отображения DataFrame

# Путь к нашему демонстрационному CFG файлу
cfg_file_to_load = os.path.join(sample_data_dir, "sample_osc_1.cfg")

try:
    osc = Oscillogram(cfg_file_to_load)
    print(f"Осциллограмма успешно загружена: {osc.filepath}")
    print(f"  File Hash (MD5 of DAT): {osc.file_hash}") 
    print(f"  Станция: {osc.station_name}")
    print(f"  Устройство: {osc.rec_dev_id}")
    print(f"  Частота: {osc.frequency} Гц")
    print(f"  Время начала записи: {osc.start_timestamp}")
    print(f"  Аналоговые каналы: {osc.analog_channel_ids}")
    print(f"  Дискретные каналы: {osc.status_channel_ids}")

    print("\nПервые 5 строк данных в виде DataFrame:")
    if pd.__version__ > '1.0.0':
        try:
            from IPython.display import display
            display(osc.data_frame.head())
        except ImportError:
            print(osc.data_frame.head())
    else:
        print(osc.data_frame.head())

except FileNotFoundError as e:
    print(f"Ошибка: Файл не найден. {e}")
except RuntimeError as e:
    print(f"Ошибка во время выполнения при загрузке осциллограммы: {e}")
except ImportError as e:
    print(f"Ошибка импорта: {e}. Убедитесь, что pandas и IPython установлены, если ошибка связана с ними.")
except Exception as e:
    print(f"Произошла неожиданная ошибка: {e}")

## 2. Использование `DataAnonymizer`

Класс `DataAnonymizer` из `preprocessing.anonymizer` предназначен для удаления конфиденциальной информации (имя станции, устройства, даты) из файлов COMTRADE и переименования файлов на основе хеш-суммы их `.dat` части.

In [ ]:
from preprocessing.anonymizer import DataAnonymizer

anonymizer = DataAnonymizer()

# Анонимизируем всю директорию
print(f"\nЗапуск анонимизации для директории: {sample_data_dir}")
# DataAnonymizer.anonymize_directory по умолчанию is_print_message=False, 
# но его внутренний метод anonymize_file печатает ошибки/предупреждения.
anonymizer.anonymize_directory(sample_data_dir) 

print("\nСодержимое директории после анонимизации:")
new_cfg_name = None
for item in os.listdir(sample_data_dir):
    print(f"- {item}")
    if item.endswith(".cfg"):
        new_cfg_name = item # Сохраняем имя для последующей загрузки

error_log_path = os.path.join(sample_data_dir, "protected_files.txt")
if os.path.exists(error_log_path):
    print("\nНайден лог ошибок 'protected_files.txt':")
    with open(error_log_path, "r", encoding="utf-8") as f:
        print(f.read())
else:
    print("\nЛог ошибок 'protected_files.txt' не создан (ошибок не было или они не были записаны).Р")

if new_cfg_name:
    print(f"\nЗагрузка анонимизированного файла: {new_cfg_name}")
    try:
        anonymized_osc = Oscillogram(os.path.join(sample_data_dir, new_cfg_name))
        print(f"  Станция (аноним.): '{anonymized_osc.station_name}'") 
        print(f"  Устройство (аноним.): '{anonymized_osc.rec_dev_id}'") 
        print(f"  Время начала записи (аноним.): {anonymized_osc.start_timestamp}") 
        print(f"  Хеш файла (должен совпадать с именем файла без .cfg): {anonymized_osc.file_hash}")
        print("\nДанные анонимизированного файла (первые 5 строк):")
        if pd.__version__ > '1.0.0':
            try:
                from IPython.display import display
                display(anonymized_osc.data_frame.head())
            except ImportError:
                print(anonymized_osc.data_frame.head())
        else:
            print(anonymized_osc.data_frame.head())
    except Exception as e:
        print(f"Ошибка при загрузке анонимизированного файла: {e}")
else:
    print("\nНе найден .cfg файл после анонимизации для демонстрации загрузки.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(sample_data_dir):
        shutil.rmtree(sample_data_dir)
        print(f"\nВременная директория {sample_data_dir} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {sample_data_dir}: {e}")